In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')

# ~
#Default Ending

import time

import lda
import lda.datasets
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from string import punctuation
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from pythainlp.corpus import thai_stopwords

stop_words = list(thai_stopwords())

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


print('OK !')

OK !


In [2]:
# !pip install pyldavis

In [3]:
def canonicalize(string):
    normalized_tokens = list()
    a = word_tokenize(string, engine = 'newmm')
    for j in a:
        token = j.strip()
        #Add clean statement here 
        if len(token) > 1 and token not in set(punctuation) and token not in ['..','...','ๆๆ']:
            try:
                normalized_tokens.append(token.lower())
            except:
                normalized_tokens.append(token)
                pass
    return normalized_tokens

In [4]:
data = pd.read_csv('data/clean_sample_text.csv')

In [5]:
# We pass in our tokenizer to the vectorizer object.

tfidf_vectorizer = TfidfVectorizer(tokenizer=canonicalize, sublinear_tf=True, stop_words = stop_words)
%time dtm_tfidf = tfidf_vectorizer.fit_transform(data['content'])

/opt/anaconda3/envs/py37_text_sum/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['กระไร', 'กาลนาน', 'ชิ้น', 'ดังที่', 'ดี', 'ดีกว่า', 'ด้อย', 'ตัว', 'ต่อไป', 'ถัดไป', 'ทั่วถึง', 'ทำ', 'ที่จะ', 'ท่าน', 'ท้าย', 'นา', 'บอ', 'บัด', 'ระยะเวลา', 'ล่ะ', 'วันวาน', 'สม', 'สมบูรณ์', 'สํา', 'หน้า', 'หรับ', 'หา', 'อย', 'เกี่ยว', 'เก่า', 'เดี๋ยวนี้', 'เย็น', 'เล่า', 'เสมือน', 'เหมือนกัน', 'แด่', 'แม้น', 'แหล่', 'โง้น', 'โน้น', 'ใด', 'ไว', 'ไหม'] not in stop_words.
  % sorted(inconsistent)


CPU times: user 707 ms, sys: 2.94 ms, total: 710 ms
Wall time: 709 ms


In [6]:
dtm_tfidf.toarray().shape

(2924, 8309)

In [7]:
tf_vectorizer = CountVectorizer(tokenizer=canonicalize, stop_words = stop_words)
%time dtm_tf = tf_vectorizer.fit_transform(data['content'])

CPU times: user 705 ms, sys: 2.57 ms, total: 707 ms
Wall time: 706 ms


In [8]:
dtm_tf.toarray().shape

(2924, 8309)

In [9]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0, verbose = True)
%time lda_tf.fit(dtm_tf)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
CPU times: user 2.38 s, sys: 11.4 ms, total: 2.39 s
Wall time: 2.38 s


LatentDirichletAllocation(n_components=20, random_state=0, verbose=True)

In [10]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0, verbose = True)
%time lda_tfidf.fit(dtm_tfidf)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
CPU times: user 1.64 s, sys: 7.61 ms, total: 1.65 s
Wall time: 1.64 s


LatentDirichletAllocation(n_components=20, random_state=0, verbose=True)

In [11]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/opt/anaconda3/envs/py37_text_sum/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/py37_text_sum/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=                          x                      y  topics  cluster  \
topic                                                                 
13      0.07195609178660888    0.18475035905649584       1        1   
9       0.13491245612155997   -0.09738364291532568       2        1   
6       0.14973052277376295   -0.03268390535852281       3        1   
1        0.2005624273641801  -0.023710383652421146       4        1   
4       0.10240390795353554    0.04645199278095844       5        1   
3      -0.09555357087569356    0.17875915047552995       6        1   
16     -0.03263572112001761    0.02807610172619503       7        1   
8       0.09493699070303264    -0.0705693026869214       8        1   
0       0.11707893251859623   -0.05817508304956591       9        1   
15     -0.00968580868463096    0.13561309195028243      10        1   
12      -0.0698659482373779    0.05774813671287256      11        1   
7      -0.15168054026483643   -0.14488166953364712      12        1   
10    0.0038776065381013625 -0.0058894711276851995      13        1   
11     -0.06243829578948334    0.02547468042129256      14        1   
19    -0.054169348439274664      0.026251948235548      15        1   
5      -0.10124415954823554    -0.1450035950680269      16        1   
2      -0.07771829859525137   0.011620909008581383      17        1   
18     -0.08661692796773501  -0.061061077804472064      18        1   
14      -0.0707576881448551  -0.011941503921782413      19        1   
17     -0.06309262809198643   -0.04344673524938566      20        1   

                    Freq  
topic                     
13    13.168963938770558  
9      7.941414152598554  
6     7.8444674792537805  
1      7.255372034114068  
4      7.129825515845472  
3      5.752708546168725  
16     5.554723970227437  
8      5.532557989003986  
0      4.891203971460498  
15     4.867344430602256  
12     4.080796979286089  
7     3.9784034783441067  
10    3.8582238866748377  
11     3.274078513856115  
19     3.008743877434102  
5     2.6619661336858242  
2     2.5679370018678154  
18      2.28222793433111  
14    2.2073817424555227  
17     2.141658424019139  , topic_info=            Term               Freq              Total Category  logprob  \
5130         พระ              252.0              252.0  Default     30.0   
6800       เจริญ              226.0              226.0  Default     29.0   
6149        สาธุ              261.0              261.0  Default     28.0   
7428          แบ               78.0               78.0  Default     27.0   
6467    อนุโมทนา               97.0               97.0  Default     26.0   
...          ...                ...                ...      ...      ...   
3575  ความเสี่ยง  3.432239018778601 42.750070603380976  Topic20  -5.5039   
8005         ​ผม  2.267777955425706 5.5159353704304355  Topic20  -5.9183   
5774         ลอง 2.7472489016923163 27.552833134198575  Topic20  -5.7265   
6192      สินค้า  2.688659202254653 34.257624597373514  Topic20   -5.748   
4942       ปัญหา 2.6901985382605798  53.16455062648311  Topic20  -5.7475   

      loglift  
5130     30.0  
6800     29.0  
6149     28.0  
7428     27.0  
6467     26.0  
...       ...  
3575   1.3214  
8005   2.9547  
5774   1.5381  
6192   1.2987  
4942   0.8598  

[1288 rows x 6 columns], token_table=      Topic                Freq            Term
term                                           
0         3 0.05876197461886193              !!
0         4 0.05876197461886193              !!
0         5 0.35257184771317157              !!
0        10 0.05876197461886193              !!
0        14  0.4113338223320335              !!
...     ...                 ...             ...
8201      3  0.9070237623885427              ☎️
8258     20  0.4635483935296639  🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥
8293      2  0.6316402906749649             🙏🙏🙏
8293     12 0.04210935271166432             🙏🙏🙏
8293     14 0.33687482169331456             🙏🙏🙏

[3317 rows x 3 columns], R=30, l

In [12]:
# pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [13]:
# pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

In [14]:
# pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

In [15]:
topic_word = lda_tf.__dict__['components_']
# topic_word = lda_tf.__dict__['exp_dirichlet_component_']

vocab = tf_vectorizer.get_feature_names()

n_top_words = 16
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: อัตราดอกเบี้ย ตลาด ไทย อ้างอิง thor การเงิน ธุรกรรม ธนาคาร เงิน แบงก์ ประเทศ ชาติ เผยแพร่ ดิจิทัล การพัฒนา ทำ
Topic 1: ชาติ แบงก์ เศรษฐกิจ https :// ly bit ธุรกิจ นโยบาย ไทย การเงิน 2563 โควิด วิชาการ งาน ธนาคารแห่งประเทศไทย
Topic 2: ดี ​คน​ สมัคร ​ค​ หนี้ ติด โค ปล่อย กู้ วิด เหมือน เปล่า ทำ เงื่อนไข ระบาด ธนาคาร
Topic 3: ดอกเบี้ย ดอก ระบบ กู้ เงิน หนี้ บัตร หยุด ปี จ่าย ค่า ดี ชำระ เงินกู้ บ้าน ​ลด​
Topic 4: เงิน ชาติ https :// แบงก์ ระบบ bit ly ฟรี ดี คน หนี้ ผลิต ลงทะเบียน การเงิน terrabkk
Topic 5: you in account and of profit are the is ดี with interested get trusted ทะเบียน com
Topic 6: แบงก์ ชาติ ไทย ทางการเงิน ?? จุด ดัชนี ปัจจัย ก.ย. ดี หุ้น สร้าง โครงการ ลงทุน เดือน เงินเฟ้อ
Topic 7: พระ เจริญ ยืน พระองค์ ธนบัตร หยุด เกล้า กระหม่อม ไหม ทำ พี่ ขอเดชะ ชื้อ ดี บาท ผม
Topic 8: เศรษฐกิจ ไทย หดตัว เรื่อง ดีขึ้น นักท่องเที่ยว เดือน การส่งออก ปี การลงทุน https :// ล้าน สินค้า มาตรการ หมวด
Topic 9: สาธุ อนุโมทนา บุญ the ปี ไทย 19 ล้าน :// บาท สินเชื่อ กราบ ประเทศ เศรษฐกิจ htt

/opt/anaconda3/envs/py37_text_sum/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
X = dtm_tf.toarray()

doc_topic = lda_tf.transform(X)
for i in range(10):
    print("{} (top topic: {})".format(data['content'].values[i], doc_topic[i].argmax()))
    print()

โดน​ค่า ATS 14
  บาท​กับ​หุ้น​อยู่​เลย (top topic: 11)

ถ้า​ตาม​นิยาม​นี้
  การ​ผูก​บัตร​เครดิต​กับ​แอ​พ ถือว่า​เป็นการ​เปลี่ยน instrument จาก card เป็น
  mobile banking (แม้​ว่า​จะ​เป็นการ​จ่าย​แบบ credit เหมือน​เดิม)
  ถูก​ต้อง​มั้ยครับ ? (top topic: 10)

ไร้​เงินสด เครดิต​ก็​ไม่​มี
  หนี้​นอก​ระบบ​ดอก​ร้อย​ละ​ร้อย (top topic: 3)

ถั่ว​ต้ม​เลย ไร้​เงินสด
  ไร้​ของ​จะ​กิน (top topic: 18)

อยากให้ลดข้อแม้ในการเข้าถึงแหล่งเงินทุนของสถาบันการเงินต่างๆ บ้าง หนี้เสียน่าจะลดลง คนเป็นหนี้ไม่มีความสุขในการเป็นหนี้หรอกครับเพียงแต่ยังไม่สามารถชำระได้ จะรวมหนี้มาจ่ายที่เดียวเพื่อจะได้มีกำลังจ่ายได้ก็ติดเรื่องคุณสมบัติโดยเฉพาะแบลคลิสต์อีก หนี้ก็เลยเป็นหนี้เสียอย่างทุกวันนี้ (top topic: 13)

เห็น​ด้วย​ครับ (top topic: 0)

เรื่อง​บัตร​เครดิต​คิด​ดอกเบี้ย​แพง
  เช่น เรา​เป็น​หนี้ 5000 ตัด​ยอด​จ่าย​4000 อีก​1000 จ่าย​รอบ​ถัด​ไป
  ทำไม​ต้อง​คิด​ดอกเบี้ย​ทั้ง​5000 ทำ​ไม่​คิด​ดอกเบี้ย แค่​1000 บาท​ที่​เหลือ
  #ธนาคารแห่งประเทศไทย (top topic: 3)

บนโลกใบนี้ ทุกคนเกิดมา ไม่มีใครเอาเงินติดตัวมาด้วย เราทุกค